In [145]:
%run "./variables_globales.ipynb"

# Plantillas de codigo
Funciones para hacer las plantillas

In [22]:
def crear_funcion(instruccion):
    """
    Crea el template de la estructura de una función

    Parametros
    ----------
    instrucion: str
        La intruccion de voz en texto.

    Regresa
    ---------
    output: str
        Codigo generado
    recomendacion: str
        Una sugerencia o fallo
    """

    global indentacion
    global recomendacion

    # guarda los avisos o recomendaciones que el programa te hace
    recomendacion = ''

    # guarda la línea de código
    output = ''
    
    # pivote que ayuda a definir el nombre de una función
    before_keyword, keyword, after_keyword = instruccion.partition('nombre')

    # verifica que haya o esté escrita la frase "nombre"
    if len(after_keyword) == 0:
        recomendacion = f'¡No me dijiste el nombre de la función!'

    # de otro modo, si tiene nombre la función
    else:

        # obtenemos el nombre de la función por el usuario
        name_func = after_keyword.split(' ')[1]

        # verificamos si no desea poner parametros                                
        if instruccion.strip().split(' ')[-1] == name_func:
            parametros = ''

        # de otro modo, si desea una función con parámetros
        else:
            before_keyword, keyword, after_keyword = instruccion.partition('parametros')

            # verifica que si exista el nombre de los parámetros
            if len(after_keyword) == 0:
                parametros = ''
                recomendacion = f'¡No me dijiste el nombre de los parámetros!'

            # escribe como parámetros todo lo que está después de "parámetros"
            else:
                parametros = after_keyword.split(' ')[1:]
                parametros = ', '.join(parametros)

        # indenta aunque marque que detecte que no le dije parámetros
        if not recomendacion or recomendacion == '¡No me dijiste el nombre de los parámetros!':
            indentacion += 1

        # concatenación del nombre y parámetros de la función
        output = f'def {name_func}({parametros}):\n' + '\t' * indentacion + '|'
    return output

#### Prueba para la funcion "crear_funcion"

In [23]:
crear_funcion('definir función con nombre magica y parametros x y z')

'def magica(x, y, z):\n\t'

## crear_condicional 

In [146]:
import re
def encontrar_palabras(transcript,cjto_palabras):

    """
    Toma un string (en minúsculos) y un conjunto de palabras. Busca el primer match
    de cjto_palabras en transcript y particiona el string

    Parametros
    ----------
    transcript: str
        La intruccion de voz en texto ya en minúsculas.
    cjto_palabras: list(str)
        Lista de strings donde se comienza a dividir el transcript original

    Regresa
    ---------
    output: list(str)
        [antes_palabra,palabra,despues_palabra]

        antes_palabra: string que está antes de la palabra de interés (de cjto_palabras)
        palabra: string que da la palabra clave donde dividimos
        despues_palabra: string que está después de la palabra
    
    Ejemplo
    --------
    encontrar_palabras('variable india producto variable alfa',['producto','suma','menos','entre'])
    >> ['variable india','producto',' variable alfa]
    """ 
    inicio,final=list(re.finditer(r'|'.join(cjto_palabras),transcript))[0].span()
    antes_palabra=transcript[:inicio].strip()
    despues_palabra=transcript[final:].strip()
    palabra=transcript[inicio:final]
    return antes_palabra,palabra,despues_palabra

def crear_condicional(transcript):
    ''' 
    Toma el transcript de un enunciado condicional y regresa su traducción a código en Python

    Parametros
    ----------
    transcript: str
        La intruccion de voz en texto ya en minúsculas.


    Regresa
    ---------
    output: str
        Cadena con el código en python, tiene una línea al final y un pipe 
        que representa el prompt donde se seguirá escribiendo
    
    Ejemplo
    --------
    crear_condicional('mientras variable india sea menor igual a numero seis')
    >> while (i<=6):
    >>      |
    '''    
    global bloque

    keyword_mapeo={'mientras':'while','si':'if','no si':'elif'}
    antes_keyword,keyword,desp_keyword=encontrar_palabras(transcript,keyword_mapeo.keys())
    cadena=keyword_mapeo[keyword]
    bloque = keyword

    # Primera división
    condicional_mapeo={'menor estricto':'<','menor o igual':'<=','igual':'==','diferente':'!='
    ,'mayor estricto':'>','mayor o igual':'>='}
    cjto_condicional=condicional_mapeo.keys()
    antes_condicional,palabra_condicional,despues_condicional=encontrar_palabras(transcript,cjto_condicional) 
    
    
    # Buscar antes en la lista de variables
    
    a_var,var,d_var=encontrar_palabras(antes_condicional,['variable'])
    nombre_var=d_var.split(' ')[0]

    if diccionario_fonetico.get(nombre_var,False):
        nombre_var=diccionario_fonetico[nombre_var]


    cadena+=' '+nombre_var+' ' +condicional_mapeo[palabra_condicional]

    # Buscar en despues_condicional el número 

    valor=despues_condicional.split(' ')[-1]

    if dict_numeros.get(valor,False):
        valor=str(dict_numeros[valor])

    global indentacion
    indentacion+=1

    return f'{keyword_mapeo[keyword]} {nombre_var} {condicional_mapeo[palabra_condicional]} {valor}:'+'\n' +'\t'* indentacion+'|'

#### Prueba para la funcion "crear_condicional"

In [254]:
a=set('definir variable india'.split(' '))
b=set(['definir','variable'])

a.union(b)

{'definir', 'india', 'variable'}

['definir', 'variable', 'india']

In [41]:
indentacion = 0
test_frases_condicional = [
    'si variable india es menor o igual a numero seis',
    'mientras variable alpha es menor igual a numero dos'
    ]

for ai in test_frases_condicional:    
    print(crear_condicional(ai))    

if i <= 6:
	|
while alpha == 2:
		|


## crear_cadena

In [43]:
def crear_cadena(transcript):
    """
    Toma el transcript de un enunciado que contiene una cadena y regresa el código en Python.
    Para usarse cuando ya se sabe que transcript sólo es los límites de la cadena

    Parametros
    ----------
    transcript: str
        La intruccion de voz en texto ya en minúsculas.


    Regresa
    ---------
    output: list(str)
        antes_palabra:parte del transcript que va antes de las comillas
        palabra: Cadena con el código en python de las comillas y lo que está adentro
        despues_palabra:parte del transcript que va antes de las comillas
    
    Ejemplo
    --------
    crear_cadena('ejecuta print con argumentos variable India producto cadena guion cadena')[1]
    >> ['ejecuta print con argumentos variable India producto','"guion"','']
    """
    try:
        inicio,final=list(re.finditer(r"cadena (.+) cadena",transcript))[0].span()
    except:
        return ''
    antes_palabra=transcript[:inicio].strip()
    despues_palabra=transcript[final:].strip()
    palabra=list(re.finditer(r"cadena (.+) cadena",transcript))[0].group(1)
    return antes_palabra,f'"{palabra}"',despues_palabra


#### Prueba para la funcion "crear_cadena"

In [51]:
test_str_cadena=[
    'ejecuta print con argumentos variable India producto cadena guión cadena',
    'ejecuta print con argumentos cadena Hola mundo cadena'
    ]

for ti in test_str_cadena:
    print(crear_cadena(ti))

('ejecuta print con argumentos variable India producto', '"guión"', '')
('ejecuta print con argumentos', '"Hola mundo"', '')


## crear_var_existente

In [49]:
def crear_var_existente(transcript):
    """
    Toma el transcript de un enunciado que contiene la mención de una variable
     y devuelve dicha variable

    Parametros
    ----------
    transcript: str
        La intruccion de voz en texto ya en minúsculas.


    Regresa
    ---------
    output: str
        palabra: Cadena con el código en python del nombre de la variable
    
    Ejemplo
    --------
    crear_var_existente('ejecuta print con argumentos variable india producto cadena guión cadena')
    >> i
    """
    try:
        antes_var,var,desp_var=encontrar_palabras(transcript,['variable'])
    except:
        return '' 

    nombre_var=desp_var.split(' ')[0]
    if diccionario_fonetico.get(nombre_var,False):
        nombre_var=diccionario_fonetico[nombre_var]
    
    return nombre_var


#### Test crear_var_existente

In [50]:
test_str_cadena=[
    'ejecuta print con argumentos variable india producto cadena guión cadena',
    'ejecuta print con argumentos cadena Hola mundo cadea'
    ]

for ti in test_str_cadena:
    print(crear_var_existente(ti))

i



## crear_operacion

In [175]:
# TODO: Hay que ver:
    # Si es otra operación hay que llamar la función recursivamente en cada pedazo
    # 1. si es cadena
    # 2. si es otra operación. Para esto, hay que regresar error o algo así cuando no se encuentre
def crear_operacion(transcript):
    '''

    Toma el transcript de una operación binaria y la traduce a código de Python.
    Para traducir las variables que se usan en la operación binaria busca 
    si son cadenas o sólo menciones de variables usando las funciones
    crear_cadena y crear_var_existente

    Parametros
    ----------
    transcript: str
        La intruccion de voz en texto ya en minúsculas.


    Regresa
    ---------
    output: str
        Cadena con el código en python
    
    Ejemplo
    --------
    crear_operacion('variable India producto cadena guión cadena')
    >> i*'-'
    '''
    global dict_operaciones
    

    try:
        antes_op,op,desp_op=encontrar_palabras(transcript,dict_operaciones.keys())
    except:
        return '' 

    # Buscamos la información en la cadena detrás del operador
    cadena_izq=crear_var_existente(antes_op)
    try:
        cadena_izq+=f'{crear_cadena(antes_op)[1]}'
    except:
        cadena_izq+=''
        
    if len(cadena_izq)==0:
        nombre_var=antes_op.split(' ')[-1]
        if dict_numeros.get(nombre_var,False):
            nombre_var=dict_numeros[nombre_var]
        cadena_izq+=str(nombre_var)
        
    # Buscamos la información en la cadena después del operador
    cadena_der=crear_var_existente(desp_op)
    try:
        cadena_der+=f'{crear_cadena(desp_op)[1]}'
    except:
        cadena_der+=''
    
    if len(cadena_der)==0:
        nombre_var=desp_op.split(' ')[0]
        if dict_numeros.get(nombre_var,False):
            nombre_var=dict_numeros[nombre_var]
        cadena_der+=str(nombre_var)

                
    return f'{cadena_izq} {dict_operaciones[op]} {cadena_der}'

#### Test crear_operacion

In [183]:
test_str_operacion=[
    'variable india producto cadena guión cadena',
    'variable india menos variable Francis','cinco entre tres',
    'variable india mas variable alfa'
    ] 

#checar tercer caso, usando números enteros o lo que están en el diccionario

for ti in test_str_operacion:
    print(crear_operacion(ti))

i * "guión"
i - Francis
5 / 3



'i * '

## crear_llamada

In [234]:
def crear_llamada(transcript):
    """
    Toma el transcript de la llamada de una función y la convierte en código de Python
        Hace uso de las funciones que detectan operaciones, variables y comillas
        ,para cada argumento de la función

    Parametros
    ----------
    transcript: str
        La intruccion de voz en texto ya en minúsculas.


    Regresa
    ---------
    output: str
        Cadena con el código en python
    
    Ejemplo
    --------
    crear_llamada(ejecuta print con argumentos variable India producto cadena guión cadena 
                    coma cadena hola cadena')
    >> print(i*'-','hola')
    
    """
    antes_ej,ej,desp_ej=encontrar_palabras(transcript,['ejecuta'])
    funcion_nombre=desp_ej.split(' ')[0]
    # Aquí tal vez valdría la pena tener un registro de las funciones previamente definidas para
    # poder buscar en un directorio con Jaccard y no aproximar

    antes_arg,keyword,desp_arg=encontrar_palabras(desp_ej,['argumentos','parametros'])

    argumentos=desp_arg.split('coma')
    lista_cadenas=[]
    for arg in argumentos:
        arg=arg.strip()
        cadena_arg=''
        # print('arg',arg)
        # Caso cuando es operacion
        cadena_op=crear_operacion(arg)
        cadena_var=crear_var_existente(arg)
        cadena_cadena=crear_cadena(arg)
        if len(cadena_op)!=0:
            lista_cadenas.append(cadena_op)
        elif len(cadena_var)!=0:
            lista_cadenas.append(cadena_var)
        elif len(cadena_cadena)!=0:
            lista_cadenas.append(cadena_cadena[1])
        else:
            nombre_var=arg
            if dict_numeros.get(nombre_var,False):
                nombre_var=str(dict_numeros[nombre_var])
                
            lista_cadenas.append(nombre_var)

        # Caso cuando es variable
        
    cadena_final=','.join(lista_cadenas)
    cadena=f'{funcion_nombre}({cadena_final})\n'+'\t'*identacion

    return cadena



#### test crear_llamada

In [235]:
test_str_ej=[
    'ejecuta print con argumentos variable india producto cadena guión cadena coma cadena hola cadena coma cinco',
    'ejecuta print con argumentos cadena hola mundo cadena'
]
for ti in test_str_ej:
    print(crear_llamada(ti))

print(i * "guión","hola",5)
print("hola mundo")


## crear_variable

In [17]:
def crear_variable(instruccion):
    """
    Estructura:
    definir variable con nombre [nombre_variable] igual a /*objeto_basico* valor/ 

    Parametros
    ----------
    instrucion: str
        La intruccion de voz en texto.

    Regresa
    ---------
    output: str
        Codigo generado
    recomendacion: str
        Una sugerencia o fallo

    Testing
    -------
    >>> definir variable con nombre india igual a numero uno
    >>> definir variable con nombre i igual a numero 1 (int)
    >>> definir variable con nombre i igual a flotante tres punto cinco (float) 
    >>> definir variable con nombre i igual a cadena hola (string) 
    >>> definir variable con nombre i igual a lista/dic (string)   
    """     
    global indentacion

    # pivote que ayuda a definir el nombre de la variable
    before_keyword, keyword, after_keyword = instruccion.partition('nombre')
    after_keyword_list = after_keyword.strip().split(' ')
    # [india igual a numero uno]
    name_variable = after_keyword_list[0]
    
    # Como sabemos que despues del nombre va seguido de "igual a"
    tipo_dato = after_keyword_list[3]        
    #print(after_keyword_list[4:]) -> lista
    valor = tipos_datos[tipo_dato](after_keyword_list[4:])        
    
    # Verificamos si es una palabra fonetica
    if diccionario_fonetico.get(name_variable,False):
        name_variable=diccionario_fonetico[name_variable] 

    codigo_generado = f'{name_variable} = {valor}\n'+ '\t' * indentacion + '|'
    return codigo_generado

####  Prueba para la funcion "crear_variable"

In [99]:
test_frases_variables = [
'definir variable con nombre india igual a numero cuatro',
'definir variable con nombre i igual a flotante tres punto cinco',
'definir variable con nombre i igual a cadena hola mundo',
'definir variable con nombre i igual a lista',
]

In [108]:
crear_variable(test_frases_variables[0])

'i = 4\n|'

In [229]:
def asignar_variable(instruccion):
    """    
    Asigna una variable (eg. indio = indio + 1)

    Parametros
    ----------
    instrucion: str
        La intruccion de voz en texto.

    Regresa
    ---------
    output: str
        Codigo generado (indio = indio + 1)
    
    Testing
    --------
    >>>'asignar variable india con india suma uno',
    >>>'asignar variable contador con contador menos uno',
    >>>'asignar variable contador con alfa',
    >>>'asignar variable india con india',
    
    """ 
    before_keyword, keyword, after_keyword = instruccion.partition('variable')
    after_keyword_list = after_keyword.strip().split(' ')
    name_variable = after_keyword_list[0]
    start = after_keyword_list.index('con') + 1
    operacion = after_keyword_list[start:]
    if len(operacion) != 1:
        operacion_str = crear_operacion(keyword + ' ' + ' '.join(operacion))
    else:
        operacion_str = operacion[0]
        # Verificamos si es una palabra fonetica para lado derecho de la 
        # asignacion
        if diccionario_fonetico.get(operacion_str,False):
            operacion_str=diccionario_fonetico[operacion_str] 

    # Verificamos si es una palabra fonetica
    if diccionario_fonetico.get(name_variable,False):
        name_variable=diccionario_fonetico[name_variable] 

    codigo_generado = f'{name_variable} = {operacion_str}\n'+ '\t' * indentacion + '|'
    return codigo_generado

#### Testing asignar_variable

In [231]:
test_asignar_var = [
'asignar variable india con india suma uno',
'asignar variable contador con contador menos uno',
'asignar variable contador con alfa',
'asignar variable india con india',
]

asignar_variable(test_asignar_var[1])

'contador = contador - 1\n|'

# crear_for

In [189]:
def crear_for(instruccion):
    """
    Crea el template de la estructura de un ciclo for.

    Parámetros
    ----------
    instrucción: str
        La intrucción de voz en texto.

    Regresa
    ---------
    output: str
        Estructura del ciclo for
    recomendacion: str
        Una sugerencia o error
    """

    global indentacion
    global recomendacion

    vocabulario_basico = ['iteracion', 'rango']

    # verificamos si la frase cumple los requisitos
    instruccion_tokens = instruccion.strip().split(' ')

    for i in vocabulario_basico:
        try:
            instruccion_tokens.index(i)
        except:
            recomendacion = 'Parece que quieres una iteración pero no reconozco tus comandos, inténtalo de nuevo'
            return f'', recomendacion

    # guarda los avisos o recomendaciones que el programa te hace
    recomendacion = ''

    # guarda la línea de código
    output = ''
    
    # pivote que ayuda a definir el rango e iterador
    before_keyword, keyword, after_keyword = instruccion.partition('iteracion')

    if after_keyword.strip().split(' ')[1] in diccionario_fonetico:
        iterador = diccionario_fonetico[after_keyword.strip().split(' ')[1]]

    else:
        iterador = after_keyword.strip().split(' ')[1]

    before_keyword, keyword, after_keyword = instruccion.partition('rango')

    limites = []

    for i, item in enumerate(after_keyword.strip().split(' ')):
        try:
            limites.append(dict_numeros[item])
        except:
            continue

    if len(limites) == 0:
        return f'', 'No encontré los límites del rango, vuelve a intentarlo'

    elif len(limites) == 1:
        return f'for {iterador} in range({limites[-1]}):\n' + '\t' * indentacion + '|', recomendacion

    elif len(limites) == 2:
        return f'for {iterador} in range({limites[0]}, {limites[1]}):\n' + '\t' * indentacion + '|', recomendacion

    elif len(limites) >= 2:
        recomendacion = 'Me dictaste más de un número en el rango pero tomé los dos primeros'
        return f'for {iterador} in range({limites[0]}, {limites[1]}):\n' + '\t' * indentacion + '|', recomendacion

####  Prueba para la funcion "crear_for"

In [200]:
print(crear_for('ejecuta una iteracion para alfa en un rango de uno a dos')[0])
print(crear_for('ejecuta una iteracion para alfa en un rango de a dos')[0])
print(crear_for('ejecuta una iteracion para alfa en un rango dos')[0])
print(crear_for('ejecuta una iteracion para india en un rango cinco')[0])

for a in range(1, 2):
|
for a in range(2):
|
for a in range(2):
|
for i in range(5):
|


Propuestas:

- **para** alfa **en** un **rango** de uno a dos

# borrar_linea

In [244]:
def borrar_linea(contexto):
    """
    Borra la última línea dado un bloque de código creado.

    Parámetros
    ----------
    instrucción: str
        Bloque de código hasta el momento.

    Regresa
    ---------
    output: str
        Nuevo bloque de código.
    """
    pass
    

#### prueba para la función "borrar_linea"

In [243]:
cadena_prueba = 'def generica(x, y)\n\ti = x\n\tj = y'

cadena_prueba[::-1]

'y = j\t\nx = i\t\n)y ,x(acireneg fed'

### Funciones pendientes

In [20]:
def crear_clase():
    global indentacion
    # ....
    
    indentacion += 1
    pass    

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=17740a58-e960-4de3-bb66-1575286930f1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>